In [1]:
#importing the libraries
import numpy as np #for mathematical calculations
import cv2 #for face detection and other image operations
import dlib #for detection of facial landmarks ex:nose,jawline,eyes
from sklearn.cluster import KMeans #for clustering
from matplotlib import pyplot as plt
from os import walk
import pandas as pd
import math
from math import degrees
import imutils
import glob
import datetime
import shutil
import os

In [ ]:
import os
f = []
outf = []
for (path, dir, files) in os.walk(".\\INPUT\\19062431\\"):
    for filename in files:
        ext = os.path.splitext(filename)[-1]
        if ext == '.jpg':
            f.append(path + "\\" + filename)
            outf.append((".\\OUTPUT" + path.replace(".","")).replace("INPUT","RESULT") + "\\" + filename)
            try:
                os.makedirs((".\\OUTPUT" + path.replace(".","")).replace("INPUT","RESULT"))
            except:
                continue;
            
print(len(f))

In [2]:
fileList = glob.glob('./INPUT/*/*/*/*/C7.jpg')

In [36]:
len(fileList)
#fileList
    
# OUYPUY/MASK_19062431-S00-L1-E01-C7.jpg
# INPUT/19062431-S00-L1-E01-C7.jpg

NameError: name 'fileList' is not defined

In [ ]:
result = pd.DataFrame(columns=['rawImagePath','outImagePath','detectFace','faceClass','landmarkPoint'])
face_cascade_path = "./haarcascade_frontalface_default.xml"
predictor_path = "./shape_predictor_68_face_landmarks.dat"

choice1 = (239, 207, 137)
choice2 = 1
dataL3 = pd.DataFrame()
right = 1
for i in range(10000,len(fileList)):
    
    print(i)

    #load the image   
    imagepath = fileList[i]
    org_path = "./OUTPUT/ORG" + fileList[i].replace("./INPUT","").replace("\\","-")
    outPath = "./OUTPUT/FACE" + fileList[i].replace("./INPUT","").replace("\\","-")
    outPath_mask = "./OUTPUT/MASK" + fileList[i].replace("./INPUT","").replace("\\","-")

    if (os.path.isfile(outPath) == True):
        continue;
        
    #create the haar cascade for detecting face and smile
    faceCascade = cv2.CascadeClassifier(face_cascade_path)

    #create the landmark predictor
    predictor = dlib.shape_predictor(predictor_path)

    #read the image
    image = cv2.imread(imagepath)

    #resizing the image to 000 cols nd 500 rows
    image = cv2.resize(image, (500, 500)) 
    #making another copy
    original = image.copy()

    #convert the image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    #apply a Gaussian blur with a 3 x 3 kernel to help remove high frequency noise
    gauss = cv2.GaussianBlur(gray,(3,3), 0)

    #Detect faces in the image
    faces = faceCascade.detectMultiScale(
        gauss,
        scaleFactor=1.05,
        minNeighbors=5,
        minSize=(100,100),
        flags=cv2.CASCADE_SCALE_IMAGE
        )

    #Detect faces in the image
    #print("found {0} faces!".format(len(faces)) )
    
    if(len(faces) != 1):
        result = result.append({'rawImagePath': imagepath,
                                'outImagePath': "",
                                'detectFace' : len(faces),
                                'faceClass' : "",
                               'landmarkPoint' : ""}, ignore_index=True)
        continue
        
    for (x,y,w,h) in faces:
        #draw a rectangle around the faces
        cv2.rectangle(image, (x,y), (x+w,y+h), (0,255,0), 2)
        #converting the opencv rectangle coordinates to Dlib rectangle
        dlib_rect = dlib.rectangle(int(x), int(y), int(x+w), int(y+h))
        #detecting landmarks
        detected_landmarks = predictor(image, dlib_rect).parts()
        #converting to np matrix
        landmarks = np.matrix([[p.x,p.y] for p in detected_landmarks])
        #landmarks array contains indices of landmarks.
        """
        #copying the image so we can we side by side
        landmark = image.copy()
        for idx, point in enumerate(landmarks):
                pos = (point[0,0], point[0,1] )
                #annotate the positions
                cv2.putText(landmark,str(idx),pos,fontFace=cv2.FONT_HERSHEY_SIMPLEX,fontScale=0.4,color=(0,0,255) )
                #draw points on the landmark positions 
                cv2.circle(landmark, pos, 3, color=(0,255,255))

    cv2.imshow("Landmarks by DLib", landmark)
    """
    #making another copy  for showing final results
    results = original.copy()

    for (x,y,w,h) in faces:
        #draw a rectangle around the faces
        cv2.rectangle(results, (x,y), (x+w,y+h), (0,255,0), 2)
        #making temporary copy
        temp = original.copy()
        #getting area of interest from image i.e., forehead (25% of face)
        forehead = temp[y:y+int(0.25*h), x:x+w]
        rows,cols, bands = forehead.shape
        X = forehead.reshape(rows*cols,bands)
        """
        Applying kmeans clustering algorithm for forehead with 2 clusters 
        this clustering differentiates between hair and skin (thats why 2 clusters)
        """
        #kmeans
        kmeans = KMeans(n_clusters=2,init='k-means++',max_iter=300,n_init=10, random_state=0)
        y_kmeans = kmeans.fit_predict(X)
        for i in range(0,rows):
            for j in range(0,cols):
                if y_kmeans[i*cols+j]==True:
                    forehead[i][j]=[255,255,255]
                if y_kmeans[i*cols+j]==False:
                    forehead[i][j]=[0,0,0]
        #Steps to get the length of forehead
        #1.get midpoint of the forehead
        #2.travel left side and right side
        #the idea here is to detect the corners of forehead which is the hair.
        #3.Consider the point which has change in pixel value (which is hair)
        forehead_mid = [int(cols/2), int(rows/2) ] #midpoint of forehead
        lef=0 
        #gets the value of forehead point
        pixel_value = forehead[forehead_mid[1],forehead_mid[0] ]
        
        try:
            for i in range(0,cols):
                #enters if when change in pixel color is detected
                if forehead[forehead_mid[1],forehead_mid[0]-i].all()!=pixel_value.all():
                    lef=forehead_mid[0]-i
                    break;
            left = [lef,forehead_mid[1]]
            rig=0
            for i in range(0,cols):
                #enters if when change in pixel color is detected
                if forehead[forehead_mid[1],forehead_mid[0]+i].all()!=pixel_value.all():
                    rig = forehead_mid[0]+i
                    break;
            right = [rig,forehead_mid[1]]
            print(right)
        except:
            result = result.append({'rawImagePath': imagepath,
                                'outImagePath': outPath,
                                'detectFace' : len(faces),
                                'faceClass' : "ERROR",
                               'landmarkPoint' : ""}, ignore_index=True)
            right = [rig,forehead_mid[1]]

    #drawing line1 on forehead with circles
    #specific landmarks are used. 
    line1 = np.subtract(right+y,left+x)[0]
    cv2.line(results, tuple(x+left), tuple(y+right), color=(0,255,0), thickness = 2)
    cv2.putText(results,' Line 1',tuple(x+left),fontFace=cv2.FONT_HERSHEY_SIMPLEX,fontScale=1,color=(0,255,0), thickness=2)
    cv2.circle(results, tuple(x+left), 5, color=(255,0,0), thickness=-1)        

    #drawing line 2 with circles
    linepointleft = (landmarks[1,0],landmarks[1,1])
    linepointright = (landmarks[15,0],landmarks[15,1])
    line2 = np.subtract(linepointright,linepointleft)[0]
    cv2.line(results, linepointleft,linepointright,color=(0,255,0), thickness = 2)
    cv2.putText(results,' Line 2',linepointleft,fontFace=cv2.FONT_HERSHEY_SIMPLEX,fontScale=1,color=(0,255,0), thickness=2)
    cv2.circle(results, linepointleft, 5, color=(255,0,0), thickness=-1)    
    cv2.circle(results, linepointright, 5, color=(255,0,0), thickness=-1)    

    #drawing line 3 with circles
    linepointleft = (landmarks[3,0],landmarks[3,1])
    linepointright = (landmarks[13,0],landmarks[13,1])
    line3 = np.subtract(linepointright,linepointleft)[0]
    cv2.line(results, linepointleft,linepointright,color=(0,255,0), thickness = 2)
    cv2.putText(results,' Line 3',linepointleft,fontFace=cv2.FONT_HERSHEY_SIMPLEX,fontScale=1,color=(0,255,0), thickness=2)
    cv2.circle(results, linepointleft, 5, color=(255,0,0), thickness=-1)    
    cv2.circle(results, linepointright, 5, color=(255,0,0), thickness=-1)    

    #drawing line 4 with circles
    linepointbottom = (landmarks[8,0],landmarks[8,1])
    linepointtop = (landmarks[8,0],y)
    line4 = np.subtract(linepointbottom,linepointtop)[1]
    cv2.line(results,linepointtop,linepointbottom,color=(0,255,0), thickness = 2)
    cv2.putText(results,' Line 4',linepointbottom,fontFace=cv2.FONT_HERSHEY_SIMPLEX,fontScale=1,color=(0,255,0), thickness=2)
    cv2.circle(results, linepointtop, 5, color=(255,0,0), thickness=-1)    
    cv2.circle(results, linepointbottom, 5, color=(255,0,0), thickness=-1)    
    #print(line1,line2,line3,line4)

    similarity = np.std([line1,line2,line3])
    #print("similarity=",similarity)
    ovalsimilarity = np.std([line2,line4])
    #print('diam=',ovalsimilarity)

    #we use arcustangens for angle calculation
    ax,ay = landmarks[3,0],landmarks[3,1]
    bx,by = landmarks[4,0],landmarks[4,1]
    cx,cy = landmarks[5,0],landmarks[5,1]
    dx,dy = landmarks[6,0],landmarks[6,1]
    
    alpha0 = math.atan2(cy-ay,cx-ax)
    alpha1 = math.atan2(dy-by,dx-bx)
    alpha = alpha1-alpha0
    angle = abs(degrees(alpha))
    angle = 180-angle

    faceType = ""
    
    for ii in range(1):
        if similarity<10:
            if angle<160:
                faceType = "squared shape"
                faceNo = 1
                result = result.append({'rawImagePath': imagepath,
                                'outImagePath': outPath,
                                'detectFace' : len(faces),
                                'faceClass' : faceType,
                               'landmarkPoint' : landmarks}, ignore_index=True)
                break
            else:
                faceType = "round shape"
                faceNo = 2
                result = result.append({'rawImagePath': imagepath,
                                'outImagePath': outPath,
                                'detectFace' : len(faces),
                                'faceClass' : faceType,
                               'landmarkPoint' : landmarks}, ignore_index=True)
                break
                
        if line3>line1:
            if angle<160:
                faceType = "triangle shape"
                faceNo = 3
                result = result.append({'rawImagePath': imagepath,
                                'outImagePath': outPath,
                                'detectFace' : len(faces),
                                'faceClass' : faceType,
                               'landmarkPoint' : landmarks}, ignore_index=True)
                break

        if ovalsimilarity<10:
            
            faceType = "diamond shape"
            faceNo = 4
            result = result.append({'rawImagePath': imagepath,
                                'outImagePath': outPath,
                                'detectFace' : len(faces),
                                'faceClass' : faceType,
                               'landmarkPoint' : landmarks}, ignore_index=True)
            break        
        if line4 > line2:
            if angle<160:
                faceType = "rectangular"
                faceNo = 5
                result = result.append({'rawImagePath': imagepath,
                                'outImagePath': outPath,
                                'detectFace' : len(faces),
                                'faceClass' : faceType,
                               'landmarkPoint' : landmarks}, ignore_index=True)
                break;
            else:
                faceType = "oblong"
                faceNo = 6
                result = result.append({'rawImagePath': imagepath,
                                'outImagePath': outPath,
                                'detectFace' : len(faces),
                                'faceClass' : faceType,
                               'landmarkPoint' : landmarks}, ignore_index=True)
                break;

        print("Damn! Contact the developer")

    #output = np.concatenate((results), axis=1)
    plt.imshow(results)
    plt.title(faceType)
    #plt.savefig(outPath)
    plt.savefig(outPath, dpi=600, bbox_inches='tight')
    
    

    ############################################################################################################################################
    img = cv2.imread(imagepath)
    img = imutils.resize(img, width = 500)
    gray=cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    #  initialization dlib Face detector 
    detector = dlib.get_frontal_face_detector()
    """
    Detect faces in grayscale images and create an object - Stores a list of coordinates for the bounding rectangle
    In the second parameter “1” Indicates that the image should be sampled up 1 Time . 
    This makes the image bigger , And allow us to detect more faces
    """
    faces = detector(gray, 1)
    #  Print the coordinates of the bounding rectangle 
    print(faces)
    print("Number of faces detected: ", len(faces))
    
    if( len(faces) != 1):
        continue
        
    """
    #  Use for Loop to extract specific coordinates (x1,x2,y1,y2)
    for face in faces:
      x1 = face.left()
      y1 = face.top()
      x2 = face.right()
      y2 = face.bottom()
      #  Draw a rectangle around the detected face
      cv2.rectangle(img, (x1,y1), (x2,y2),(0,255,0),3)
    cv2.imshow("image", img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    """
    
    #  File path 
    p = "shape_predictor_68_face_landmarks.dat"

    predictor = dlib.shape_predictor(p)
    #  Use predictors to get shapes 
    for face in faces:
        landmarks = predictor(gray, face)

    points = []
    
    try:
        for i in range(1, 16):
            point = [landmarks.part(i).x, landmarks.part(i).y]
            points.append(point)

            # print(points).
            #  wide , High coverage masks 
            mask_a = [((landmarks.part(42).x), (landmarks.part(15).y)),
                      ((landmarks.part(27).x), (landmarks.part(27).y)),
                      ((landmarks.part(39).x), (landmarks.part(1).y))]
            #  wide , In the middle of the mask 
            mask_c = [((landmarks.part(29).x), (landmarks.part(29).y))]
            #  wide 、 Low coverage masks 
            mask_e = [((landmarks.part(35).x), (landmarks.part(35).y)),
                      ((landmarks.part(34).x), (landmarks.part(34).y)),
                      ((landmarks.part(33).x), (landmarks.part(33).y)),
                      ((landmarks.part(32).x), (landmarks.part(32).y)),
                      ((landmarks.part(31).x), (landmarks.part(31).y))]
            fmask_a = points + mask_a
            fmask_c = points + mask_c
            fmask_e = points + mask_e
            # mask_type = {1: fmask_a, 2: fmask_c, 3: fmask_e}
            # mask_type[choice2]
            #  Use Python OpenCV - cv2.polylines() Method is [mask_type] Draw the mask outline :
            # fmask_a = wide, high coverage mask,
            # fmask_c = wide, medium coverage mask,
            # fmask_e  = wide, low coverage mask
            fmask_a = np.array(fmask_a, dtype=np.int32)
            fmask_c = np.array(fmask_c, dtype=np.int32)
            fmask_e = np.array(fmask_e, dtype=np.int32)
            mask_type = {1: fmask_a, 2: fmask_c, 3: fmask_e}
            mask_type[choice2]
            #  Change parameters [mask_type] and color_type For various combinations 
            img2 = cv2.polylines(img, [mask_type[choice2]], True, choice1, thickness=2, lineType=cv2.LINE_8)
            #  Use Python OpenCV - cv2.fillPoly() How to fill the mask 
            #  Change parameters [mask_type] and color_type For various combinations 
            img3 = cv2.fillPoly(img2, [mask_type[choice2]], choice1, lineType=cv2.LINE_AA)
    except:
        continue
        
    # cv2.imshow("image with mask outline", img2)
    cv2.imshow("image with mask", img3)
    #  Save the output file for the test 
    outputNameofImage = "imagetest2_2_b.jpg"
    print("Saving output image to", outputNameofImage)
    cv2.imwrite(outPath_mask, img3)
    
    shutil.copyfile( imagepath , org_path )
    
    
    dict = {
              'FACE_NO': [faceNo]
            , 'FACE_TYPE': [faceType]
            , 'IMG_PATH': [imagepath]
            , 'MASK_YN': ['N']
            , 'IMG_HOR_SIZE': [img.shape[0]]
            , 'IMG_VER_SIZE': [img.shape[1]]
            , 'IMAGE_EXT': ["jpg"]
            , 'line1': [line1]
            , 'line2': [line2]
            , 'line3': [line3]
            , 'line4': [line4]
            , 'regtime' : [str(datetime.datetime.now())]
            , 'modtime' : [str(datetime.datetime.now())]
            , 'masktype' : [choice2]
        }

    dataL3 = dataL3.append(pd.DataFrame.from_dict(dict))
    #plt.imshow(outputNameofImage)
    #plt.show()
    
dataL3.to_excel("RESULT.xlsx")

In [48]:
df2 = pd.read_csv("./SERVICE5_RESULT_DATA.csv")

In [49]:
del df2["NO"]

In [50]:
#result = df2.drop_duplicates()
result = df2
print(type(result))
result


<class 'pandas.core.frame.DataFrame'>


,FACE_NO,FACE_TYPE,MASK_YN,IMG_HOR_SIZE,IMG_VER_SIZE,IMG_EXT,FRHD_LINE,CNTR_LINE,PHLT_LINE,CHIN_LINE,REG_DATE,MOD_DATE,MASK_TYPE,ORG_IMG_URL,FACE_IMG_URL,MASK_IMG_URL,FACE_COORD,MASK_COORD
0,6,oblong,N,332,500,jpg,38,142,123,193,2021-09-10,2021-09-10,1,http://210.178.19.87:9080/IMG/SERVICE5/ORG-190...,http://210.178.19.87:9080/IMG/SERVICE5/FACE-19...,http://210.178.19.87:9080/IMG/SERVICE5/MASK-19...,"[(192, 157), (193, 173), (195, 188), (197, 202...",[[193 173] [195 188] [197 202] [202 215] [210 ...
1,6,oblong,N,332,500,jpg,73,127,110,192,2021-09-10,2021-09-10,1,http://210.178.19.87:9080/IMG/SERVICE5/ORG-190...,http://210.178.19.87:9080/IMG/SERVICE5/FACE-19...,http://210.178.19.87:9080/IMG/SERVICE5/MASK-19...,"[(189, 162), (191, 179), (194, 194), (197, 208...",[[191 179] [194 194] [197 208] [203 221] [212 ...
2,6,oblong,N,332,500,jpg,78,141,121,191,2021-09-10,2021-09-10,1,http://210.178.19.87:9080/IMG/SERVICE5/ORG-190...,http://210.178.19.87:9080/IMG/SERVICE5/FACE-19...,http://210.178.19.87:9080/IMG/SERVICE5/MASK-19...,"[(189, 162), (189, 178), (191, 193), (195, 207...",[[189 178] [191 193] [195 207] [200 220] [209 ...
3,6,oblong,N,332,500,jpg,43,146,128,191,2021-09-10,2021-09-10,1,http://210.178.19.87:9080/IMG/SERVICE5/ORG-190...,http://210.178.19.87:9080/IMG/SERVICE5/FACE-19...,http://210.178.19.87:9080/IMG/SERVICE5/MASK-19...,"[(192, 155), (192, 171), (194, 186), (196, 200...",[[192 171] [194 186] [196 200] [201 213] [209 ...
4,6,oblong,N,332,500,jpg,76,146,126,190,2021-09-10,2021-09-10,1,http://210.178.19.87:9080/IMG/SERVICE5/ORG-190...,http://210.178.19.87:9080/IMG/SERVICE5/FACE-19...,http://210.178.19.87:9080/IMG/SERVICE5/MASK-19...,"[(188, 160), (190, 177), (193, 193), (196, 207...",[[190 177] [193 193] [196 207] [203 220] [212 ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20577,6,oblong,N,332,500,jpg,84,116,105,190,2021-09-14,2021-09-14,1,http://210.178.19.87:9080/IMG/SERVICE5/ORG-191...,http://210.178.19.87:9080/IMG/SERVICE5/FACE-19...,http://210.178.19.87:9080/IMG/SERVICE5/MASK-19...,"[(196, 146), (196, 161), (197, 177), (199, 191...",[[196 161] [197 177] [199 191] [202 205] [209 ...
20578,6,oblong,N,332,500,jpg,32,118,106,177,2021-09-14,2021-09-14,1,http://210.178.19.87:9080/IMG/SERVICE5/ORG-191...,http://210.178.19.87:9080/IMG/SERVICE5/FACE-19...,http://210.178.19.87:9080/IMG/SERVICE5/MASK-19...,"[(193, 149), (195, 164), (198, 179), (200, 193...",[[195 164] [198 179] [200 193] [204 206] [210 ...
20579,6,oblong,N,332,500,jpg,31,131,119,172,2021-09-14,2021-09-14,1,http://210.178.19.87:9080/IMG/SERVICE5/ORG-191...,http://210.178.19.87:9080/IMG/SERVICE5/FACE-19...,http://210.178.19.87:9080/IMG/SERVICE5/MASK-19...,"[(193, 150), (195, 166), (197, 181), (201, 195...",[[195 166] [197 181] [201 195] [207 209] [216 ...
20580,6,oblong,N,332,500,jpg,68,149,133,194,2021-09-14,2021-09-14,1,http://210.178.19.87:9080/IMG/SERVICE5/ORG-191...,http://210.178.19.87:9080/IMG/SERVICE5/FACE-19...,http://210.178.19.87:9080/IMG/SERVICE5/MASK-19...,"[(192, 153), (193, 167), (195, 180), (198, 194...",[[193 167] [195 180] [198 194] [203 206] [211 ...


In [51]:
result.index.names = ['NO']
result

,FACE_NO,FACE_TYPE,MASK_YN,IMG_HOR_SIZE,IMG_VER_SIZE,IMG_EXT,FRHD_LINE,CNTR_LINE,PHLT_LINE,CHIN_LINE,REG_DATE,MOD_DATE,MASK_TYPE,ORG_IMG_URL,FACE_IMG_URL,MASK_IMG_URL,FACE_COORD,MASK_COORD
NO,,,,,,,,,,,,,,,,,,
0,6,oblong,N,332,500,jpg,38,142,123,193,2021-09-10,2021-09-10,1,http://210.178.19.87:9080/IMG/SERVICE5/ORG-190...,http://210.178.19.87:9080/IMG/SERVICE5/FACE-19...,http://210.178.19.87:9080/IMG/SERVICE5/MASK-19...,"[(192, 157), (193, 173), (195, 188), (197, 202...",[[193 173] [195 188] [197 202] [202 215] [210 ...
1,6,oblong,N,332,500,jpg,73,127,110,192,2021-09-10,2021-09-10,1,http://210.178.19.87:9080/IMG/SERVICE5/ORG-190...,http://210.178.19.87:9080/IMG/SERVICE5/FACE-19...,http://210.178.19.87:9080/IMG/SERVICE5/MASK-19...,"[(189, 162), (191, 179), (194, 194), (197, 208...",[[191 179] [194 194] [197 208] [203 221] [212 ...
2,6,oblong,N,332,500,jpg,78,141,121,191,2021-09-10,2021-09-10,1,http://210.178.19.87:9080/IMG/SERVICE5/ORG-190...,http://210.178.19.87:9080/IMG/SERVICE5/FACE-19...,http://210.178.19.87:9080/IMG/SERVICE5/MASK-19...,"[(189, 162), (189, 178), (191, 193), (195, 207...",[[189 178] [191 193] [195 207] [200 220] [209 ...
3,6,oblong,N,332,500,jpg,43,146,128,191,2021-09-10,2021-09-10,1,http://210.178.19.87:9080/IMG/SERVICE5/ORG-190...,http://210.178.19.87:9080/IMG/SERVICE5/FACE-19...,http://210.178.19.87:9080/IMG/SERVICE5/MASK-19...,"[(192, 155), (192, 171), (194, 186), (196, 200...",[[192 171] [194 186] [196 200] [201 213] [209 ...
4,6,oblong,N,332,500,jpg,76,146,126,190,2021-09-10,2021-09-10,1,http://210.178.19.87:9080/IMG/SERVICE5/ORG-190...,http://210.178.19.87:9080/IMG/SERVICE5/FACE-19...,http://210.178.19.87:9080/IMG/SERVICE5/MASK-19...,"[(188, 160), (190, 177), (193, 193), (196, 207...",[[190 177] [193 193] [196 207] [203 220] [212 ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20577,6,oblong,N,332,500,jpg,84,116,105,190,2021-09-14,2021-09-14,1,http://210.178.19.87:9080/IMG/SERVICE5/ORG-191...,http://210.178.19.87:9080/IMG/SERVICE5/FACE-19...,http://210.178.19.87:9080/IMG/SERVICE5/MASK-19...,"[(196, 146), (196, 161), (197, 177), (199, 191...",[[196 161] [197 177] [199 191] [202 205] [209 ...
20578,6,oblong,N,332,500,jpg,32,118,106,177,2021-09-14,2021-09-14,1,http://210.178.19.87:9080/IMG/SERVICE5/ORG-191...,http://210.178.19.87:9080/IMG/SERVICE5/FACE-19...,http://210.178.19.87:9080/IMG/SERVICE5/MASK-19...,"[(193, 149), (195, 164), (198, 179), (200, 193...",[[195 164] [198 179] [200 193] [204 206] [210 ...
20579,6,oblong,N,332,500,jpg,31,131,119,172,2021-09-14,2021-09-14,1,http://210.178.19.87:9080/IMG/SERVICE5/ORG-191...,http://210.178.19.87:9080/IMG/SERVICE5/FACE-19...,http://210.178.19.87:9080/IMG/SERVICE5/MASK-19...,"[(193, 150), (195, 166), (197, 181), (201, 195...",[[195 166] [197 181] [201 195] [207 209] [216 ...


In [52]:
result['NO'] = result.index
result

,FACE_NO,FACE_TYPE,MASK_YN,IMG_HOR_SIZE,IMG_VER_SIZE,IMG_EXT,FRHD_LINE,CNTR_LINE,PHLT_LINE,CHIN_LINE,REG_DATE,MOD_DATE,MASK_TYPE,ORG_IMG_URL,FACE_IMG_URL,MASK_IMG_URL,FACE_COORD,MASK_COORD,NO
NO,,,,,,,,,,,,,,,,,,,
0,6,oblong,N,332,500,jpg,38,142,123,193,2021-09-10,2021-09-10,1,http://210.178.19.87:9080/IMG/SERVICE5/ORG-190...,http://210.178.19.87:9080/IMG/SERVICE5/FACE-19...,http://210.178.19.87:9080/IMG/SERVICE5/MASK-19...,"[(192, 157), (193, 173), (195, 188), (197, 202...",[[193 173] [195 188] [197 202] [202 215] [210 ...,0
1,6,oblong,N,332,500,jpg,73,127,110,192,2021-09-10,2021-09-10,1,http://210.178.19.87:9080/IMG/SERVICE5/ORG-190...,http://210.178.19.87:9080/IMG/SERVICE5/FACE-19...,http://210.178.19.87:9080/IMG/SERVICE5/MASK-19...,"[(189, 162), (191, 179), (194, 194), (197, 208...",[[191 179] [194 194] [197 208] [203 221] [212 ...,1
2,6,oblong,N,332,500,jpg,78,141,121,191,2021-09-10,2021-09-10,1,http://210.178.19.87:9080/IMG/SERVICE5/ORG-190...,http://210.178.19.87:9080/IMG/SERVICE5/FACE-19...,http://210.178.19.87:9080/IMG/SERVICE5/MASK-19...,"[(189, 162), (189, 178), (191, 193), (195, 207...",[[189 178] [191 193] [195 207] [200 220] [209 ...,2
3,6,oblong,N,332,500,jpg,43,146,128,191,2021-09-10,2021-09-10,1,http://210.178.19.87:9080/IMG/SERVICE5/ORG-190...,http://210.178.19.87:9080/IMG/SERVICE5/FACE-19...,http://210.178.19.87:9080/IMG/SERVICE5/MASK-19...,"[(192, 155), (192, 171), (194, 186), (196, 200...",[[192 171] [194 186] [196 200] [201 213] [209 ...,3
4,6,oblong,N,332,500,jpg,76,146,126,190,2021-09-10,2021-09-10,1,http://210.178.19.87:9080/IMG/SERVICE5/ORG-190...,http://210.178.19.87:9080/IMG/SERVICE5/FACE-19...,http://210.178.19.87:9080/IMG/SERVICE5/MASK-19...,"[(188, 160), (190, 177), (193, 193), (196, 207...",[[190 177] [193 193] [196 207] [203 220] [212 ...,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20577,6,oblong,N,332,500,jpg,84,116,105,190,2021-09-14,2021-09-14,1,http://210.178.19.87:9080/IMG/SERVICE5/ORG-191...,http://210.178.19.87:9080/IMG/SERVICE5/FACE-19...,http://210.178.19.87:9080/IMG/SERVICE5/MASK-19...,"[(196, 146), (196, 161), (197, 177), (199, 191...",[[196 161] [197 177] [199 191] [202 205] [209 ...,20577
20578,6,oblong,N,332,500,jpg,32,118,106,177,2021-09-14,2021-09-14,1,http://210.178.19.87:9080/IMG/SERVICE5/ORG-191...,http://210.178.19.87:9080/IMG/SERVICE5/FACE-19...,http://210.178.19.87:9080/IMG/SERVICE5/MASK-19...,"[(193, 149), (195, 164), (198, 179), (200, 193...",[[195 164] [198 179] [200 193] [204 206] [210 ...,20578
20579,6,oblong,N,332,500,jpg,31,131,119,172,2021-09-14,2021-09-14,1,http://210.178.19.87:9080/IMG/SERVICE5/ORG-191...,http://210.178.19.87:9080/IMG/SERVICE5/FACE-19...,http://210.178.19.87:9080/IMG/SERVICE5/MASK-19...,"[(193, 150), (195, 166), (197, 181), (201, 195...",[[195 166] [197 181] [201 195] [207 209] [216 ...,20579


In [53]:
result['NO'] = result['NO'] + 1


In [54]:
result

,FACE_NO,FACE_TYPE,MASK_YN,IMG_HOR_SIZE,IMG_VER_SIZE,IMG_EXT,FRHD_LINE,CNTR_LINE,PHLT_LINE,CHIN_LINE,REG_DATE,MOD_DATE,MASK_TYPE,ORG_IMG_URL,FACE_IMG_URL,MASK_IMG_URL,FACE_COORD,MASK_COORD,NO
NO,,,,,,,,,,,,,,,,,,,
0,6,oblong,N,332,500,jpg,38,142,123,193,2021-09-10,2021-09-10,1,http://210.178.19.87:9080/IMG/SERVICE5/ORG-190...,http://210.178.19.87:9080/IMG/SERVICE5/FACE-19...,http://210.178.19.87:9080/IMG/SERVICE5/MASK-19...,"[(192, 157), (193, 173), (195, 188), (197, 202...",[[193 173] [195 188] [197 202] [202 215] [210 ...,1
1,6,oblong,N,332,500,jpg,73,127,110,192,2021-09-10,2021-09-10,1,http://210.178.19.87:9080/IMG/SERVICE5/ORG-190...,http://210.178.19.87:9080/IMG/SERVICE5/FACE-19...,http://210.178.19.87:9080/IMG/SERVICE5/MASK-19...,"[(189, 162), (191, 179), (194, 194), (197, 208...",[[191 179] [194 194] [197 208] [203 221] [212 ...,2
2,6,oblong,N,332,500,jpg,78,141,121,191,2021-09-10,2021-09-10,1,http://210.178.19.87:9080/IMG/SERVICE5/ORG-190...,http://210.178.19.87:9080/IMG/SERVICE5/FACE-19...,http://210.178.19.87:9080/IMG/SERVICE5/MASK-19...,"[(189, 162), (189, 178), (191, 193), (195, 207...",[[189 178] [191 193] [195 207] [200 220] [209 ...,3
3,6,oblong,N,332,500,jpg,43,146,128,191,2021-09-10,2021-09-10,1,http://210.178.19.87:9080/IMG/SERVICE5/ORG-190...,http://210.178.19.87:9080/IMG/SERVICE5/FACE-19...,http://210.178.19.87:9080/IMG/SERVICE5/MASK-19...,"[(192, 155), (192, 171), (194, 186), (196, 200...",[[192 171] [194 186] [196 200] [201 213] [209 ...,4
4,6,oblong,N,332,500,jpg,76,146,126,190,2021-09-10,2021-09-10,1,http://210.178.19.87:9080/IMG/SERVICE5/ORG-190...,http://210.178.19.87:9080/IMG/SERVICE5/FACE-19...,http://210.178.19.87:9080/IMG/SERVICE5/MASK-19...,"[(188, 160), (190, 177), (193, 193), (196, 207...",[[190 177] [193 193] [196 207] [203 220] [212 ...,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20577,6,oblong,N,332,500,jpg,84,116,105,190,2021-09-14,2021-09-14,1,http://210.178.19.87:9080/IMG/SERVICE5/ORG-191...,http://210.178.19.87:9080/IMG/SERVICE5/FACE-19...,http://210.178.19.87:9080/IMG/SERVICE5/MASK-19...,"[(196, 146), (196, 161), (197, 177), (199, 191...",[[196 161] [197 177] [199 191] [202 205] [209 ...,20578
20578,6,oblong,N,332,500,jpg,32,118,106,177,2021-09-14,2021-09-14,1,http://210.178.19.87:9080/IMG/SERVICE5/ORG-191...,http://210.178.19.87:9080/IMG/SERVICE5/FACE-19...,http://210.178.19.87:9080/IMG/SERVICE5/MASK-19...,"[(193, 149), (195, 164), (198, 179), (200, 193...",[[195 164] [198 179] [200 193] [204 206] [210 ...,20579
20579,6,oblong,N,332,500,jpg,31,131,119,172,2021-09-14,2021-09-14,1,http://210.178.19.87:9080/IMG/SERVICE5/ORG-191...,http://210.178.19.87:9080/IMG/SERVICE5/FACE-19...,http://210.178.19.87:9080/IMG/SERVICE5/MASK-19...,"[(193, 150), (195, 166), (197, 181), (201, 195...",[[195 166] [197 181] [201 195] [207 209] [216 ...,20580


In [28]:
result.insert(0, 'NO', np.arange(len(result)))

In [56]:
result['NO'] = result['NO'] + 1
result

,FACE_NO,FACE_TYPE,MASK_YN,IMG_HOR_SIZE,IMG_VER_SIZE,IMG_EXT,FRHD_LINE,CNTR_LINE,PHLT_LINE,CHIN_LINE,REG_DATE,MOD_DATE,MASK_TYPE,ORG_IMG_URL,FACE_IMG_URL,MASK_IMG_URL,FACE_COORD,MASK_COORD,NO
NO,,,,,,,,,,,,,,,,,,,
0,6,oblong,N,332,500,jpg,38,142,123,193,2021-09-10,2021-09-10,1,http://210.178.19.87:9080/IMG/SERVICE5/ORG-190...,http://210.178.19.87:9080/IMG/SERVICE5/FACE-19...,http://210.178.19.87:9080/IMG/SERVICE5/MASK-19...,"[(192, 157), (193, 173), (195, 188), (197, 202...",[[193 173] [195 188] [197 202] [202 215] [210 ...,1
1,6,oblong,N,332,500,jpg,73,127,110,192,2021-09-10,2021-09-10,1,http://210.178.19.87:9080/IMG/SERVICE5/ORG-190...,http://210.178.19.87:9080/IMG/SERVICE5/FACE-19...,http://210.178.19.87:9080/IMG/SERVICE5/MASK-19...,"[(189, 162), (191, 179), (194, 194), (197, 208...",[[191 179] [194 194] [197 208] [203 221] [212 ...,2
2,6,oblong,N,332,500,jpg,78,141,121,191,2021-09-10,2021-09-10,1,http://210.178.19.87:9080/IMG/SERVICE5/ORG-190...,http://210.178.19.87:9080/IMG/SERVICE5/FACE-19...,http://210.178.19.87:9080/IMG/SERVICE5/MASK-19...,"[(189, 162), (189, 178), (191, 193), (195, 207...",[[189 178] [191 193] [195 207] [200 220] [209 ...,3
3,6,oblong,N,332,500,jpg,43,146,128,191,2021-09-10,2021-09-10,1,http://210.178.19.87:9080/IMG/SERVICE5/ORG-190...,http://210.178.19.87:9080/IMG/SERVICE5/FACE-19...,http://210.178.19.87:9080/IMG/SERVICE5/MASK-19...,"[(192, 155), (192, 171), (194, 186), (196, 200...",[[192 171] [194 186] [196 200] [201 213] [209 ...,4
4,6,oblong,N,332,500,jpg,76,146,126,190,2021-09-10,2021-09-10,1,http://210.178.19.87:9080/IMG/SERVICE5/ORG-190...,http://210.178.19.87:9080/IMG/SERVICE5/FACE-19...,http://210.178.19.87:9080/IMG/SERVICE5/MASK-19...,"[(188, 160), (190, 177), (193, 193), (196, 207...",[[190 177] [193 193] [196 207] [203 220] [212 ...,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20577,6,oblong,N,332,500,jpg,84,116,105,190,2021-09-14,2021-09-14,1,http://210.178.19.87:9080/IMG/SERVICE5/ORG-191...,http://210.178.19.87:9080/IMG/SERVICE5/FACE-19...,http://210.178.19.87:9080/IMG/SERVICE5/MASK-19...,"[(196, 146), (196, 161), (197, 177), (199, 191...",[[196 161] [197 177] [199 191] [202 205] [209 ...,20578
20578,6,oblong,N,332,500,jpg,32,118,106,177,2021-09-14,2021-09-14,1,http://210.178.19.87:9080/IMG/SERVICE5/ORG-191...,http://210.178.19.87:9080/IMG/SERVICE5/FACE-19...,http://210.178.19.87:9080/IMG/SERVICE5/MASK-19...,"[(193, 149), (195, 164), (198, 179), (200, 193...",[[195 164] [198 179] [200 193] [204 206] [210 ...,20579
20579,6,oblong,N,332,500,jpg,31,131,119,172,2021-09-14,2021-09-14,1,http://210.178.19.87:9080/IMG/SERVICE5/ORG-191...,http://210.178.19.87:9080/IMG/SERVICE5/FACE-19...,http://210.178.19.87:9080/IMG/SERVICE5/MASK-19...,"[(193, 150), (195, 166), (197, 181), (201, 195...",[[195 166] [197 181] [201 195] [207 209] [216 ...,20580


In [57]:
result.to_csv("./result.csv",index=False)
result.to_excel("./result.xlsx",index=False)